# Data Engineer Certification Sample Practical Exam

HappyPaws, creates fun and educational apps for pet owners. 

HappyPaws wants to help pet owners understand their pets better by tracking their activities and health through the app. 

The data engineering team is responsible for making sure all the pet data from thousands of users is organized and safe, so pet owners can get tips to keep their pets happy and healthy.



## Task

HappyPaws has collected three datasets over the past year: 
 - "pet_activities.csv" which logs daily activities of pets, 
 - "pet_health.csv" which records vet visits and health issues, and 
 - "users.csv" which contains information about the pet owners. 

Each dataset contains unique identifiers for pets and/or their owners. 

The engineers developing the app currently write code to cross reference all of these data sources. 

They want to make things easier by having a single table with all data included.

Your manager has asked you to write a Python function that cleans and merges these datasets into a single dataset. 

The final dataset should provide a comprehensive view of each pet's activities, health records, and owner information. 

- To test your code, your manager will run only the code `all_pet_data('pet_activities.csv', 'pet_health.csv', 'users.csv')`
- Your `all_pet_data()` function must return a DataFrame, with columns as described below.
- All columns must accurately match the descriptions provided below, including names.

## Data

The data that has been provided has the following structure

![image](image.png)

The function that you write must return data as described below. There should be a unique row for each activity/health visit. 

Where missing values are permitted, they should be in the default Python format.

|Column Name | Description |
|-------|-------|
| pet_id | Unique identifier for each pet. There should not be any missing values.|
| date | The date of the activity recorded or the date of the health visit, in date format. There should not be any missing values.|
| activity_type | The type of activity, one of 'Walking', 'Playing', 'Resting' or for rows that relate to a health visit, the value 'Health'. Missing values are permitted.|
| duration_minutes | The duration of the activity in minutes. For rows that relate to health visits, this should be 0. Missing values for other activities are permitted.|
| issue | The health issue identified or check-up note. For rows that relate to activities, this should be a missing value. Missing values for health activities are permitted.|
| resolution | The outcome or advice given for the issue. For rows that relate to activities, this should be a missing value. Missing values for health activities are permitted.|
| owner_id | Unique identifier for the pet owner. All pets must have an owner.|
| owner_age_group | The age group of the owner (e.g., 18-25, 26-35, etc.). Missing values are permitted.|
| pet_type | The type of pet (e.g., Dog, Cat). Missing values are permitted.|

In [1]:
import pandas as pd
import numpy as np
import datetime

def all_pet_data(activities,pet_health,users):
    
    pet_activities_df = pd.read_csv(activities)
    pet_health_df = pd.read_csv(pet_health)
    users_df = pd.read_csv(users)
    
    # Check Pet ID for unique values 
    if users_df['pet_id'].duplicated().any():
        raise ValueError("pet_id in users_df should be unique")

    pet_activities_df['duration_minutes'] = pd.to_numeric(pet_activities_df['duration_minutes'].replace('-', 0), errors='coerce')
    

    # Check for null values in pet_id and date 
    pet_activities_df = pet_activities_df.dropna(subset=['pet_id', 'date'])
    pet_health_df = pet_health_df.dropna(subset=['pet_id', 'visit_date'])
    users_df = users_df.dropna(subset=['pet_id', 'owner_id'])

    #pet_activities_df data cleaning 
    pet_activities_df['date']= pd.to_datetime(pet_activities_df.date)

    pet_activities_df['activity_type'] = pet_activities_df.activity_type.replace({'Walk':'Walking','Play':'Playing','Rest':'Resting'})

    pet_activities_df['activity_type'] = pet_activities_df.activity_type.astype('category')

    pet_activities_df['duration_minutes'] = pd.to_numeric(pet_activities_df.duration_minutes.replace('-',np.nan),errors = 'coerce')
 

    pet_activities_df['issue'] = np.nan
    
    pet_activities_df['resolution'] = np.nan


    pet_activities_df = pet_activities_df.drop_duplicates(['pet_id', 'date'])
    
    

    #pet_health_df data cleaning 
    pet_health_df['visit_date'] = pd.to_datetime(pet_health_df['visit_date'])

    pet_health_df = pet_health_df.rename(columns={'visit_date':'date'})

    pet_health_df['activity_type'] = 'Health'
    
    pet_health_df['activity_type'] = pet_health_df.activity_type.astype('category')
    

    pet_health_df['duration_minutes'] = 0

    pet_health_df['issue'] = pet_health_df.issue.astype('category')

    pet_health_df['resolution'] = pet_health_df.resolution.str.capitalize()


    pet_health_df = pet_health_df.drop_duplicates(['pet_id', 'date'])

    #users_df data cleaning
    users_df['owner_age_group'] = users_df.owner_age_group.astype('category')

    users_df['pet_type'] = users_df.pet_type.astype('category')
    

    # Set the columns  
    activity_cols = ['pet_id', 'date', 'activity_type', 'duration_minutes', 'issue', 'resolution']
    
    pet_activities_df = pet_activities_df[activity_cols]
    
    pet_health_df = pet_health_df[activity_cols]

    # concat pet_Activities_Df and pet_healt_df

    df_merged = pd.concat([pet_activities_df,pet_health_df], ignore_index = True)

    #merge df_merged with users_df
    
    all_data = df_merged.merge(users_df, how='inner', on='pet_id')

    final_columns = [
        'pet_id', 'date', 'activity_type', 'duration_minutes',
        'issue', 'resolution', 'owner_id', 'owner_age_group', 'pet_type'
    ]

    
    all_data = all_data[final_columns]

    
    #validation checks
    
    if all_data['pet_id'].isnull().any():
        raise ValueError('Missing values found in pet_id')
        
    if all_data['date'].isnull().any():
        raise ValueError('Missing values found in date')
        
    if all_data['owner_id'].isnull().any():
        raise ValueError('Missing values found in owner_id')
        
    valid_activities = {'Walking', 'Playing', 'Resting', 'Health'}
    if not all_data['activity_type'].isin(valid_activities).all():
        raise ValueError('Invalid activity_type values found')
    
        
    return all_data

all_pet_data('pet_activities.csv', 'pet_health.csv', 'users.csv')                           

,pet_id,date,activity_type,duration_minutes,issue,resolution,owner_id,owner_age_group,pet_type
0,9930,2023-08-01,Resting,0,NaN,NaN,7016,18-25,Dog
1,9930,2023-05-15,Walking,0,NaN,NaN,7016,18-25,Dog
2,9930,2022-09-13,Resting,46,NaN,NaN,7016,18-25,Dog
3,9930,2023-03-16,Resting,34,NaN,NaN,7016,18-25,Dog
4,9930,2022-09-05,Health,0,Injury,"Sprain identified, rest and limited movement a...",7016,18-25,Dog
...,...,...,...,...,...,...,...,...,...
1871,2492,2023-07-28,Health,0,Annual Checkup,"Vaccination completed, next due in 1 year",7325,46-55,Cat
1872,9136,2022-04-06,Health,0,Injury,Physical therapy recommended for joint issues,3553,18-25,Dog
1873,9136,2022-04-15,Health,0,Annual Checkup,"All clear, healthy diet recommended",3553,18-25,Dog
1874,9856,2023-01-15,Health,0,Dental Cleaning,"Cleaning done, no dental issues",9424,36-45,Dog


In [60]:
all_pet_data('pet_activities.csv', 'pet_health.csv', 'users.csv').info()
all_pet_data('pet_activities.csv', 'pet_health.csv', 'users.csv').isna().sum()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1876 entries, 0 to 1875
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   pet_id            1876 non-null   int64         
 1   date              1876 non-null   datetime64[ns]
 2   activity_type     1876 non-null   object        
 3   duration_minutes  1876 non-null   int64         
 4   issue             938 non-null    category      
 5   resolution        938 non-null    object        
 6   owner_id          1876 non-null   int64         
 7   owner_age_group   1876 non-null   category      
 8   pet_type          1876 non-null   category      
dtypes: category(3), datetime64[ns](1), int64(3), object(2)
memory usage: 108.7+ KB


pet_id                0
date                  0
activity_type         0
duration_minutes      0
issue               938
resolution          938
owner_id              0
owner_age_group       0
pet_type              0
dtype: int64